In [ ]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [ ]:
class SPT():
    def __init__(self, source_node):
        # Importing all files in folder and create key/value pairs,(source_node,[(node1,dist),(node2,dist),...])
        # and persist it to reduce shuffling through network
        self.graph = sc.wholeTextFiles(r"C:\Users\taylankabbani2019\Downloads\directedgraph\*") \
                        .flatMap(lambda x: x[1].split(' \n')).map(lambda x: (x.split(' ')[0], x.split(' ')[1:])) \
                        .mapValues(lambda x: [(x[i], x[i+1]) for i in range(0, len(x), 2)]).persist()
        # Define source node
        self.src = source_node
        # Create distance RDD, initially 0 for the source and inf to other nodes
        # Also persist it.
        self.dist = self.graph.map(lambda x: (x[0], float('inf')) if x[0] != source_node else (x[0], 0))
        
    def shortest_path(self):
        #use accumulator to check the termination condition
        acc = sc.accumulator(0)
        while True:
            # Finding the shortest distance between each node and the source:
            update_dist = self.graph.join(self.dist).flatMap(lambda x: [(x[1][0][i][0] , int(x[1][0][i][1]) + x[1][1]) \
                                                                        for i in range(len(x[1][0]))] \
                                                             if x[1][1] != float('inf') else []) \
                                                    .reduceByKey(lambda x,y: min(x,y))
            
            # Update the distance if it's shorter than the one found in the previous iteration:
            self.dist = self.dist.leftOuterJoin(update_dist).mapValues(lambda x: x[0] if x[1] == None else min(x[0],x[1]))
            
            # Checking if there are nodes we did not visit yet
            self.dist.foreach(lambda node: acc.add(1) if node[1] == float('inf') else None)
            
            # If We visited all nodes terminate
            if acc.value == 0:
                break
            else:
                acc.value = 0
        return self.dist
    def col(self):
        print(self.dist.take(5))
        
t = SPT('node5089')
t.shortest_path()